In [1]:
from rlbench.environment import Environment
from rlbench.action_modes import ArmActionMode, ActionMode
from rlbench.observation_config import ObservationConfig
from rlbench.tasks import PutGroceriesInCupboard
import numpy as np
import scipy as sp
from quaternion import from_rotation_matrix, quaternion
from scipy.spatial.transform import Rotation as R
from pyrep.const import ConfigurationPathAlgorithms as Algos
import pprint
import time
import math

from utils import RandomAgent, NoisyObjectPoseSensor, VisionObjectPoseSensor, RobotController,rotation_transform
from DQN_take_object_out_class import*


/home/stefanzhu/anaconda3/envs/auto_proj2/lib/python3.6/site-packages/quaternion/numba_wrapper.py:20: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from numba, which means that some
parts of this code may run MUCH more slowly.  You
may wish to install numba.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)


In [2]:

if __name__ == "__main__":

    #Create RL agent 

    obs_config = ObservationConfig()
    obs_config.set_all(True)

    action_mode = ActionMode(ArmActionMode.DELTA_JOINT_POSITION)
    env = Environment(
        action_mode, obs_config=obs_config, headless=False)
    env.launch()

    task = env.get_task(PutGroceriesInCupboard)
    robot_controller = RobotController(env, task)

In [ ]:
    RLagent = rl_take_out()
#     RLagent.agent.load(directory="model")

    training_steps = 120000
    episode_length = 20
    save_freq = 10000
    obs = None
    
    #Getting the reset point position
    reset_position, reset_orientation, reset_obj = robot_controller.get_pose_and_object_from_simulation("target_waypoint")
    
    for i in range(training_steps):
        if i % episode_length == 0:
            
            descriptions, obs = task.reset()

            #(1) Go to waypoint3 outside of cupboard
            next_position, next_orientation, _ = robot_controller.get_pose_and_object_from_simulation("waypoint3")
            try:
                robot_controller.move(next_position, next_orientation, 1)
            except:
                continue 
            waypoint3_orient = next_orientation
            #(2) Reach to the spam and close gripper
            tar_init_pos, tar_verticle_orientation, target_obj = robot_controller.get_pose_and_object_from_simulation("spam_grasp_point")
            print("target obj")
            try:
                path = robot_controller.move(tar_init_pos, waypoint3_orient,1 ,ignore_collisions=True)
            except:
                continue
            robot_controller.actuate_gripper(0)
            robot_controller. (np.flip(path,0),0)
        
        #Getting grasp success state 
        grasp_success = robot_controller.get_grasp_state()
        
        # Getting the updated target position and orientation 
        tar_position = list(target_obj.get_position())
        tar_orientation = list(target_obj.get_orientation())
        #Getting the arm pose and
        arm_pose = robot_controller.get_arm_tip_pose()  
        
        state = tar_position + tar_orientation
        
        action = RLagent.get_action(state)
        
#         action = [45,45,90,45,45,45,45,45]
        
        action = np.array(action)
        action = action*0.01
        action = list(action)
        
        obs, _, _ = task.step(action)
#         time.sleep(0.5)
        
        
        reward, terminal = RLagent.get_reward(tar_position, reset_position, tar_orientation, tar_verticle_orientation, tar_init_pos,grasp_success)
#         print(reward)
        print("random action", np.around(action, decimals=1))

        RLagent.agent.observe(terminal=terminal, reward=reward)
        
        
        if i % save_freq == save_freq - 1: RLagent.agent.save(directory="model")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Planning, with collision checks
Moving to desired position, orientation and gripper state
planned action [9.498570534560713e-07, 0.17560794949531555, 9.05998513189843e-06, -0.8732151985168457, -3.5589646358857863e-06, 1.2215098142623901, 0.7853855490684509, 1]
planned action [0.008039328269660473, 0.11115675419569016, 0.0022758888080716133, -0.9863748550415039, -0.011035573668777943, 1.2968791723251343, 0.7972879409790039, 1]
planned action [0.014797267504036427, 0.05967345088720322, 0.00555097172036767, -1.0760998725891113, -0.021884039044380188, 1.361525297164917, 0.8108698129653931, 1]
planned action [0.020625397562980652, 0.015147664584219456, 0.009557091630995274, -1.153419852256775, -0.03293677046895027, 1.4205149412155151, 0.8250681161880493, 1]
planned action [0.02574111521244049,

planned action [-0.661038875579834, -0.8480063080787659, 0.7297702431678772, -2.3733339309692383, -1.0461715459823608, 2.8638901710510254, 2.357006788253784, 1]
planned action [-0.696194589138031, -0.8797860741615295, 0.7545567154884338, -2.3781208992004395, -1.065854787826538, 2.878260374069214, 2.4090309143066406, 1]
target obj
Planning, not checking for collisions...
Moving to desired position, orientation and gripper state
planned action [-1.7240405082702637, -1.761934757232666, 2.632824420928955, -3.0708374977111816, -2.8973000049591064, 3.7521231174468994, 2.8969736099243164, 1]
planned action [-1.7263470888137817, -1.7578986883163452, 2.631129741668701, -3.065211534500122, -2.8922295570373535, 3.7493982315063477, 2.896109104156494, 1]
planned action [-1.7286537885665894, -1.753862738609314, 2.6294350624084473, -3.0595853328704834, -2.8871593475341797, 3.746673345565796, 2.8952443599700928, 1]
planned action [-1.7309603691101074, -1.7498266696929932, 2.6277403831481934, -3.053959

planned action [-1.8393720388412476, -1.5601332187652588, 2.5480921268463135, -2.7895336151123047, -2.643782138824463, 3.615877389907837, 2.853742837905884, 1]
planned action [-1.8416787385940552, -1.556097149848938, 2.5463976860046387, -2.783907651901245, -2.63871169090271, 3.613152503967285, 2.8528780937194824, 1]
planned action [-1.8439853191375732, -1.5520612001419067, 2.5447030067443848, -2.7782814502716064, -2.633641481399536, 3.6104276180267334, 2.85201358795166, 1]
planned action [-1.8462920188903809, -1.548025131225586, 2.543008327484131, -2.772655487060547, -2.628571033477783, 3.6077027320861816, 2.851148843765259, 1]
planned action [-1.848598599433899, -1.5439890623092651, 2.541313648223877, -2.767029285430908, -2.6235005855560303, 3.604977607727051, 2.8502843379974365, 1]
planned action [-1.850905179977417, -1.5399531126022339, 2.539618968963623, -2.7614033222198486, -2.6184303760528564, 3.602252721786499, 2.849419593811035, 1]
planned action [-1.8532118797302246, -1.535917

planned action [-1.9593168497085571, -1.3502596616744995, 2.459970712661743, -2.4969775676727295, -2.3801236152648926, 3.474181890487671, 2.8087825775146484, 1]
planned action [-1.9616235494613647, -1.3462235927581787, 2.4582760334014893, -2.49135160446167, -2.3750531673431396, 3.471457004547119, 2.807918071746826, 1]
planned action [-1.9639301300048828, -1.342187523841858, 2.4565813541412354, -2.4857254028320312, -2.3699827194213867, 3.4687318801879883, 2.807053327560425, 1]
planned action [-1.9662368297576904, -1.3381515741348267, 2.4548866748809814, -2.4800994396209717, -2.364912509918213, 3.4660069942474365, 2.8061888217926025, 1]
planned action [-1.9685434103012085, -1.3341155052185059, 2.4531922340393066, -2.474473237991333, -2.35984206199646, 3.4632821083068848, 2.805324077606201, 1]
planned action [-1.9708499908447266, -1.330079436302185, 2.4514975547790527, -2.4688472747802734, -2.354771614074707, 3.460557222366333, 2.804459571838379, 1]
planned action [-1.9731566905975342, -1

planned action [-2.0792617797851562, -1.1403859853744507, 2.371849298477173, -2.2044215202331543, -2.116464853286743, 3.332486152648926, 2.763822555541992, 1]
planned action [-2.0815682411193848, -1.1363500356674194, 2.370154619216919, -2.1987955570220947, -2.1113946437835693, 3.329761266708374, 2.762957811355591, 1]
planned action [-2.0838749408721924, -1.1323139667510986, 2.368459939956665, -2.193169355392456, -2.1063241958618164, 3.3270363807678223, 2.7620933055877686, 1]
planned action [-2.086181640625, -1.1282778978347778, 2.366765260696411, -2.1875433921813965, -2.1012537479400635, 3.3243114948272705, 2.761228561401367, 1]
planned action [-2.0884883403778076, -1.124241828918457, 2.3650705814361572, -2.181917190551758, -2.0961835384368896, 3.3215866088867188, 2.760364055633545, 1]
planned action [-2.090794801712036, -1.1202058792114258, 2.3633759021759033, -2.1762912273406982, -2.0911130905151367, 3.318861722946167, 2.7594995498657227, 1]
planned action [-2.0931015014648438, -1.11

...

planned action [-1.9400537014007568, -1.5921891927719116, 2.624805212020874, -2.6483380794525146, -2.468380928039551, 3.6266658306121826, 2.701617479324341, 1]
planned action [-1.9438649415969849, -1.5892003774642944, 2.624340772628784, -2.6409192085266113, -2.4608561992645264, 3.6244606971740723, 2.6983206272125244, 1]
planned action [-1.9476763010025024, -1.5862114429473877, 2.6238765716552734, -2.633500576019287, -2.453331232070923, 3.622255802154541, 2.695023536682129, 1]
planned action [-1.95148766040802, -1.5832226276397705, 2.6234121322631836, -2.626081705093384, -2.4458062648773193, 3.6200506687164307, 2.6917266845703125, 1]
planned action [-1.955298900604248, -1.5802336931228638, 2.6229476928710938, -2.6186630725860596, -2.438281536102295, 3.6178457736968994, 2.688429832458496, 1]
planned action [-1.9591102600097656, -1.5772448778152466, 2.622483491897583, -2.6112442016601562, -2.4307565689086914, 3.615640640258789, 2.6851329803466797, 1]
planned action [-1.9629216194152832, -

planned action [-2.138242483139038, -1.436767578125, 2.6006581783294678, -2.262561082839966, -2.0770864486694336, 3.51200532913208, 2.5301780700683594, 1]
planned action [-2.1420536041259766, -1.4337786436080933, 2.600193738937378, -2.2551424503326416, -2.06956148147583, 3.509800434112549, 2.526881217956543, 1]
planned action [-2.145864963531494, -1.430789828300476, 2.599729299545288, -2.2477235794067383, -2.0620367527008057, 3.5075953006744385, 2.5235841274261475, 1]
planned action [-2.1496763229370117, -1.4278008937835693, 2.5992650985717773, -2.240304946899414, -2.054511785507202, 3.5053904056549072, 2.520287275314331, 1]
planned action [-2.1534876823425293, -1.4248120784759521, 2.5988006591796875, -2.2328860759735107, -2.0469868183135986, 3.503185272216797, 2.5169904232025146, 1]
planned action [-2.157299041748047, -1.4218231439590454, 2.5983362197875977, -2.2254672050476074, -2.039462089538574, 3.5009803771972656, 2.5136935710906982, 1]
planned action [-2.1611104011535645, -1.4188

planned action [-2.3364310264587402, -1.2813458442687988, 2.5765109062194824, -1.876784324645996, -1.6857919692993164, 3.3973450660705566, 2.358738660812378, 1]
planned action [-2.340242385864258, -1.278356909751892, 2.5760467052459717, -1.8693654537200928, -1.6782671213150024, 3.3951401710510254, 2.3554418087005615, 1]
planned action [-2.3440537452697754, -1.275368094444275, 2.575582265853882, -1.861946702003479, -1.670742154121399, 3.392935037612915, 2.352144718170166, 1]
planned action [-2.347865104675293, -1.2723791599273682, 2.575117826461792, -1.8545279502868652, -1.663217306137085, 3.390730142593384, 2.3488478660583496, 1]
planned action [-2.3516764640808105, -1.269390344619751, 2.5746536254882812, -1.847109079360962, -1.6556923389434814, 3.3885250091552734, 2.345551013946533, 1]
planned action [-2.355487585067749, -1.2664014101028442, 2.5741891860961914, -1.8396903276443481, -1.6481674909591675, 3.386320114135742, 2.342254161834717, 1]
planned action [-2.3592989444732666, -1.26

planned action [-2.5346198081970215, -1.1259241104125977, 2.552363872528076, -1.4910073280334473, -1.2944974899291992, 3.282684803009033, 2.1872992515563965, 1]
planned action [-2.538431167602539, -1.122935175895691, 2.5518994331359863, -1.4835885763168335, -1.2869725227355957, 3.280479907989502, 2.18400239944458, 1]
planned action [-2.5422422885894775, -1.1199463605880737, 2.5514352321624756, -1.4761698246002197, -1.2794476747512817, 3.2782747745513916, 2.1807053089141846, 1]
planned action [-2.546053647994995, -1.116957426071167, 2.5509707927703857, -1.4687509536743164, -1.2719227075576782, 3.2760698795318604, 2.177408456802368, 1]
planned action [-2.5498650074005127, -1.1139686107635498, 2.550506353378296, -1.4613322019577026, -1.2643978595733643, 3.27386474609375, 2.1741116046905518, 1]
planned action [-2.5536763668060303, -1.110979676246643, 2.550042152404785, -1.4539134502410889, -1.2568730115890503, 3.2716598510742188, 2.1708147525787354, 1]
planned action [-2.557487726211548, -

planned action [-2.7328083515167236, -0.9705023765563965, 2.52821683883667, -1.105230450630188, -0.9032029509544373, 3.1680245399475098, 2.015859842300415, 1]
planned action [-2.736619710922241, -0.9675135016441345, 2.52775239944458, -1.0978116989135742, -0.8956780433654785, 3.1658194065093994, 2.0125629901885986, 1]
planned action [-2.740431070327759, -0.9645246267318726, 2.5272879600524902, -1.090392827987671, -0.8881531357765198, 3.163614511489868, 2.009265899658203, 1]
planned action [-2.7442424297332764, -0.9615357518196106, 2.5268237590789795, -1.0829740762710571, -0.880628228187561, 3.161409378051758, 2.0059690475463867, 1]
planned action [-2.748053789138794, -0.9585468769073486, 2.5263593196868896, -1.0755553245544434, -0.8731033205986023, 3.1592044830322266, 2.0026721954345703, 1]
planned action [-2.7518651485443115, -0.9555580019950867, 2.5258948802948, -1.0681365728378296, -0.8655784726142883, 3.156999349594116, 1.9993752241134644, 1]
planned action [-2.75567626953125, -0.95

In [48]:
action = [math.pi/2,math.pi/2,math.pi/2,math.pi/2,math.pi/2,math.pi/2,math.pi/2,math.pi/2]

In [49]:
# descriptions, obs = task.reset()
obs, _, _ = task.step(action)